In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import csv
import numpy as np

DATA_DIR = 'MNIST'
sentiment_data = 'sentiment-data'


def getWordVectorDict():
    reader = csv.reader(open(sentiment_data +'/word-vectors-refine.txt'))

    word_vector_dict = {}
    for row in reader:
        key = row[0]
        if key in word_vector_dict:
            # implement your duplicate row handling here
            pass
        word_vector_dict[key] = np.array(row[1:])
    return word_vector_dict

def getPaddedSentenceMatrix(sentenceMatrix):
    wordCount = 100
    return np.vstack((sentenceMatrix, np.zeros((wordCount - np.shape(sentenceMatrix)[0],np.shape(sentenceMatrix)[1]), dtype=np.float32)))

def getVectorForSentence(sentence, word_vec_dict):
    sentence_matrix = []
    for word in sentence.split(' '):
        word_vec = word_vec_dict[word]
        if(len(sentence_matrix) == 0):
            sentence_matrix = word_vec
        else:
            sentence_matrix = np.vstack((sentence_matrix,word_vec))
    return getPaddedSentenceMatrix(sentence_matrix)

def getData(fileName):
    reader = csv.reader(open(sentiment_data +'/' + fileName))
    trainingData = []
    for row in reader:
        data = {}
        data['label'] =  1 if row[0] == 'postive' else 0
        data['sentence'] = row[1:]
        trainingData.append(data)
    return trainingData

word_vec_dict = getWordVectorDict()


def transform(row):
    return row['label'], getVectorForSentence(row['sentence'][0], word_vec_dict)


word_vector_size = 50;
time_steps = 100;
num_classes = 2
batch_size = 1000;
n_iterations = 10;
hidden_layer_size = 64

training_data = getData('train.csv')
training_rows  = map(lambda row: transform(row), training_data)
training_data = map(lambda row: row[1], training_rows)
training_labels = map(lambda row: row[0], training_rows)
#test_data = map(lambda row: transform(row), getData('test.csv'))
test_data = getData('test.csv')
test_rows  = map(lambda row: transform(row), test_data)
test_data = map(lambda row: row[1], test_rows)
test_labels = map(lambda row: row[0], test_rows)
print training_data[0]

[['0.68491' '0.32385' '-0.11592' ..., '0.17874' '-0.1693' '0.062375']
 ['0.96193' '0.012516' '0.21733' ..., '0.14032' '-0.38468' '-0.38712']
 ['0.6008' '0.18044' '0.078339' ..., '-0.016404' '-0.65372' '-0.38255']
 ..., 
 ['0.88387' '-0.14199' '0.13566' ..., '0.52711' '-0.20148' '0.0095952']
 ['-0.0010919' '0.33324' '0.35743' ..., '-0.45697' '-0.048969' '1.1316']
 ['-0.55114' '-0.16296' '-0.95494' ..., '-1.0346' '-0.25143' '1.4836']]


In [2]:

#Setting up the input and labels placeholders
_inputs = tf.placeholder(tf.float32, shape=[None, time_steps,
                                            word_vector_size])
y = tf.placeholder(tf.int32, shape=[None, num_classes])
y_one_hot = tf.one_hot( y , num_classes )

# TensorFlow built-in functions
# Creating the RNN cell and creating the outputs
with tf.variable_scope("gru"):
    gru_cell = tf.contrib.rnn.GRUCell(hidden_layer_size)
    outputs, states = tf.nn.dynamic_rnn(gru_cell,_inputs, dtype=tf.float32)
    final_output = tf.layers.dense(states, num_classes)



softmax = tf.nn.softmax_cross_entropy_with_logits(logits = final_output,
                                                  labels = y)                         
cross_entropy = tf.reduce_mean(softmax)
optimizer = tf.train.RMSPropOptimizer(0.001, 0.9)
gvs = optimizer.compute_gradients(cross_entropy)
capped_gvs = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gvs]
train_step = optimizer.apply_gradients(capped_gvs)
correct_prediction = tf.equal(tf.argmax(y,1),
                              tf.argmax(final_output,1))
accuracy = (tf.reduce_mean(tf.cast(correct_prediction,
                                   tf.float32)))*100






In [3]:
def dense_to_one_hot(labels_dense, num_classes=10):
  """Convert class labels from scalars to one-hot vectors."""
  num_labels = labels_dense.shape[0]
  index_offset = np.arange(num_labels) * num_classes
  labels_one_hot = np.zeros((num_labels, num_classes))
  labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
  return labels_one_hot

label_batch = np.array(training_labels[100 : 200])
l = np.array(dense_to_one_hot(label_batch,2)).reshape(len(label_batch), num_classes)

print l.shape

(100, 2)


In [4]:
batch_size = 100;
n_iterations = 20;

#Initialize session
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for epoch in range(n_iterations):
        print "Epoch:", epoch
        for j in range(len(training_data)/batch_size):
            #print "j:", j
            if(j == 100):
                break
            startIndex = j*batch_size
            endIndex = startIndex + batch_size
            batch_x = np.array(training_data[startIndex : endIndex]).reshape((-1,time_steps, word_vector_size))
            batch_y = dense_to_one_hot(np.array(training_labels[startIndex : endIndex]),num_classes)
            sess.run(train_step, feed_dict={_inputs: batch_x,
                                            y: batch_y})
        if epoch % 1 == 0:
            acc = sess.run(accuracy, feed_dict={_inputs: batch_x, y: batch_y})
            loss = sess.run(cross_entropy, feed_dict={_inputs: batch_x, y: batch_y})
            print ("Iter " + str(epoch) + ", Minibatch Loss= " + \
                    "{:.6f}".format(loss) + ", Training Accuracy= " + \
                    "{:.5f}".format(acc))
            
            _states = np.array(sess.run(states, feed_dict={_inputs: batch_x, y: batch_y}))
            _outputs = np.array(sess.run(outputs, feed_dict={_inputs: batch_x, y: batch_y}))
      
            final_op = _outputs[-1][-1]
            #final_op = final_op[-1]
            print (_states.shape)
            print (_outputs.shape)
            print (_states[-1])
            print (final_op)
            print (final_op.shape)


    print ("Testing Accuracy:",
        sess.run(accuracy, feed_dict={_inputs: test_data, y: dense_to_one_hot(np.array(test_labels), num_classes)}))

Epoch: 0
Iter 0, Minibatch Loss= 0.697023, Training Accuracy= 50.00000
(100, 64)
(100, 100, 64)
[ 0.5162347  -0.11819154 -0.5463233   0.5598706  -0.19427732 -0.25532776
  0.11919738  0.66415358  0.24820271 -0.6127857   0.409504   -0.15468313
  0.73689514 -0.49753702  0.00544862 -0.22898878  0.40075496  0.08938645
  0.06321375 -0.09962279  0.18321204  0.33886611 -0.35042232 -0.86028928
  0.55071449 -0.22403556 -0.11768804 -0.5925293   0.57846802  0.44640893
  0.01750622 -0.02580878  0.12711397 -0.47312716  0.32164839  0.40020624
 -0.65026784  0.05934783 -0.50766468  0.29480997 -0.20243013 -0.38786352
  0.6475563  -0.59787196 -0.59995413 -0.66767967 -0.39042392 -0.26720676
 -0.46581605  0.59464693  0.19762833 -0.53131968 -0.55406857 -0.41526851
 -0.35170147 -0.70137882  0.05803543  0.05674836  0.62532026  0.24105331
  0.27033341  0.27540094  0.50771028  0.39567053]
[ 0.5162347  -0.11819154 -0.5463233   0.5598706  -0.19427732 -0.25532776
  0.11919738  0.66415358  0.24820271 -0.6127857   0

KeyboardInterrupt: 

The accuracies for the RNN, LSTM and GRU are 70.7%, 78.08% and 77.26% respectively.

In [ ]:
weights = {
        'linear_layer': tf.Variable(tf.truncated_normal([hidden_layer_size,
                                                         num_classes],
                                                         mean=0,stddev=.01))
}


# Extract the last relevant output and use in a linear layer
final_output = tf.matmul(states,
                         weights["linear_layer"])